In [ ]:
# to automatically reload modules who's content has changed
%load_ext autoreload
%autoreload 2

# configure matplotlib
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

In [ ]:
from task_utils import *

In [ ]:
import funbo as fb
import funbo.plotting as fp

In [ ]:
domain_bounds = [('x1', 0, 6), ('x2', 0, 6)]
range_bounds = (-1, 2.5)

In [ ]:
noisy = False
def to_fit(x, y):
    ''' from https://github.com/fmfn/BayesianOptimization/issues/18 '''
    a = np.exp(-( (x - 2)**2/0.7 + (y - 4)**2/1.2) + (x - 2)*(y - 4)/1.6 )
    b = np.exp(-( (x - 4)**2/3 + (y - 2)**2/2.) )
    c = np.exp(-( (x - 4)**2/0.5 + (y - 4)**2/0.5) + (x - 4)*(y - 4)/0.5 )
    d = np.sin(3.1415 * x)
    e = np.exp(-( (x - 5.5)**2/0.5 + (y - 5.5)**2/.5) )
    val = 2*a + b - c + 0.17 * d + 2*e
    return val + np.random.normal(0, 0.2, size=None if isinstance(x, float) else x.shape) if noisy else val

In [ ]:
def plot_to_fit():
    g = fb.utils.RegularGrid(50, [b[1:] for b in domain_bounds])
    X, Y = g.meshgrid()
    Z = to_fit(X, Y)
    fp.surface_3D(X, Y, Z, axes_names=('x1', 'x2', 'y'))
plot_to_fit()

In [ ]:
def make_objective(to_fit, sample_num, sample_dist):
    def objective(f):
        bounds = [b[1:] for b in domain_bounds]
        # global reward
        R_g = fp.multidimensional_integrate(lambda x: (f(x) - to_fit(x))**2, bounds)
        # local rewards
        R_ls = []
        if sample_dist == 'linear':
            reward_xs = np.linspace(xmin, xmax, num=sample_num)
        elif sample_dist == 'random':
            reward_xs = np.random.uniform(xmin, xmax, size=(sample_num,))
        else:
            raise ValueError()
        for x in reward_xs:
            R_l = (f(x)-to_fit(x))**2
            R_ls.append((x, R_l))
        return R_ls, R_g
    return objective

In [ ]:
f = fb.utils.InterpolatedFunction(g.get_points(), np.array(values).reshape(-1,1))

In [ ]:
X, Y = eval_g.meshgrid()
Z = f(eval_g.get_points()).reshape(X.shape)
fp.surface_3D(X, Y, Z)

In [ ]:
def to_fit(x):
    return np.sin(x) * 2*np.cos(x/4)
g = fb.utils.RegularGrid(10, [(0, 10)])

In [ ]:
xs = g.get_points()
ys = to_fit(xs)

In [ ]:
f = scipy.interpolate.interp1d(xs.flatten(), ys.flatten(), kind='quadratic')

In [ ]:
g = fb.utils.RegularGrid(100, [(0, 10)])
plt.plot(g.get_points(), f(g.get_points().flatten()))